In [ ]:
!pip install --upgrade pip
!pip install torch_geometric
!pip install torch
!pip install matplotlib

In [19]:
!pip list
!pip install -U executing

ERROR! Session/line number was not unique in database. History logging moved to new session 45
Package                    Version
-------------------------- ----------
absl-py                    1.4.0
aiohttp                    3.9.1
aiosignal                  1.3.1
appnope                    0.1.3
asttokens                  2.4.1
astunparse                 1.6.3
attrs                      23.1.0
blinker                    1.6.2
CacheControl               0.13.1
cachetools                 5.3.1
certifi                    2023.11.17
cffi                       1.15.1
charset-normalizer         3.1.0
click                      8.1.3
cloud-sql-python-connector 1.4.3
cloudevents                1.9.0
comm                       0.1.4
contourpy                  1.2.0
cryptography               41.0.1
cycler                     0.12.1
debugpy                    1.8.0
decorator                  5.1.1
deprecation                2.1.0
executing                  2.0.1
filelock                   3.1

In [20]:
import torch 
from torch_geometric.nn import GCNConv, SAGEConv, to_hetero

from torch_geometric.utils import negative_sampling

from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

from sklearn.metrics import roc_auc_score

device = torch.device("cpu")

transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, 
                      num_test=0.1, # uses 10% of data for test data set, rest is for training
                      is_undirected=True, 
                      add_negative_train_samples=False)
])

# data
dataset = Planetoid(root="testdata", name="Cora", transform=transform)


# print("Data Info: ", dataset.name)
# print("Number of graphs: ", len(dataset))
# print("Number of features: ", dataset.num_features)
# print("Number of classes (node types): ", dataset.num_classes)
# print("Number of edge features: ", dataset.num_edge_features)
# print("Number of node features: ", dataset.num_node_features )


print(dataset.get_summary)

train_data, val_data, test_data = dataset[0]
dataset[0]

print(len(train_data), len(val_data), len(test_data))


<bound method Dataset.get_summary of Cora()>
8 8 8


In [22]:
# GCNN Model Implementation

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    # forward pass (embeddings)
    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    
    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)
    
    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()
    

model = Net(dataset.num_features, 128, 64)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index,
        num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1),
        method='sparse'
    )

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1
    )

    edge_label = torch.cat(
        [train_data.edge_label, train_data.edge_label.new_zeros(neg_edge_index.size(1))],
        dim=0
    )

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss

In [24]:
@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


In [25]:
best_val_auc = final_test_auc = 0
for epoch in range(1,101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, ' f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')
z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

Epoch: 001, Loss: 0.6930, Val: 0.6472, Test: 0.7210
Epoch: 002, Loss: 0.6823, Val: 0.6366, Test: 0.7165
Epoch: 003, Loss: 0.7088, Val: 0.6434, Test: 0.7220
Epoch: 004, Loss: 0.6774, Val: 0.6642, Test: 0.7341
Epoch: 005, Loss: 0.6849, Val: 0.6824, Test: 0.7420
Epoch: 006, Loss: 0.6882, Val: 0.6964, Test: 0.7455
Epoch: 007, Loss: 0.6890, Val: 0.6867, Test: 0.7383
Epoch: 008, Loss: 0.6885, Val: 0.6675, Test: 0.7255
Epoch: 009, Loss: 0.6864, Val: 0.6518, Test: 0.7155
Epoch: 010, Loss: 0.6820, Val: 0.6390, Test: 0.7093
Epoch: 011, Loss: 0.6759, Val: 0.6346, Test: 0.7055
Epoch: 012, Loss: 0.6720, Val: 0.6305, Test: 0.7007
Epoch: 013, Loss: 0.6753, Val: 0.6309, Test: 0.6952
Epoch: 014, Loss: 0.6714, Val: 0.6375, Test: 0.6920
Epoch: 015, Loss: 0.6621, Val: 0.6531, Test: 0.6959
Epoch: 016, Loss: 0.6597, Val: 0.6727, Test: 0.7068
Epoch: 017, Loss: 0.6567, Val: 0.6872, Test: 0.7165
Epoch: 018, Loss: 0.6512, Val: 0.6902, Test: 0.7160
Epoch: 019, Loss: 0.6434, Val: 0.6890, Test: 0.7134
Epoch: 020, 

In [26]:
final_edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [   0,    2,    4,  ..., 2705, 2706, 2707]])

In [27]:
# Built own dataset

import numpy as np
import networkx as nx
from torch_geometric.utils.convert import to_networkx
from torch_geometric.transforms import ToUndirected
import torch_geometric.data as dt
import matplotlib.pyplot as plt
import h5py

import scipy.sparse as sp

# make a graph with 100 nodes, and 16 dimensions per node (node features)
# randomizes feature vectors with values (0-1)
nodes = torch.rand((100, 16), dtype=torch.float) 


# randomly sample indexes of 100 nodes and creates 500 edges
rows = np.random.choice(100, 250)
cols = np.random.choice(100, 250)
edges = torch.tensor([rows, cols])

# randomly adds 500 edge attributes between 0 and 3 (a random attribute for each edge)
edges_attr = np.random.choice(3,250)

# generates 100 0's or 1's (for nodes), long means categotical data
ys = torch.rand((100)).round().long()


# Convert graph information to PyG data object:
'''
Data(x, edge_index, edges_attr, y)

    x ~ Node feature matrix --> [num_nodes x num_node_features]

    edge_index ~ Graph connectivity in COO format with shape [2, num_edges]

    edges_attr ~ similar to x but for edges; [num_edges, num_edge_features]

    y ~ Graph level or node level ground-truth labels with arbitrary shape; node_labels

'''


graph = dt.HeteroData(x=nodes, edge_index=edges, edges_attr=edges_attr, y=ys)

# vis = to_networkx(graph)
# node_labels = graph.y.numpy()
# plt.figure(1, figsize=(15, 13))
# nx.draw(vis, cmap=plt.get_cmap('Set3'), node_color=node_labels, node_size=70, linewidths=6)
# plt.show()

In [28]:
# Load data from .mat files

gf_data = h5py.File("../data/GeneFeatures.mat", 'r')
gp_data= h5py.File("../data/genes_phenes.mat", 'r')
cf_data = h5py.File("../data/clinicalfeatures_tfidf.mat", 'r')

# print(cf_data.keys())

gene_ids = gp_data["geneIds"]
phene_ids = gp_data["pheneIds"]

# GeneGene_Hs --> "data", "ir", "jc"
print("GeneGene data: ", gp_data["GeneGene_Hs"]["data"].shape)
print("GeneGene ir: ", gp_data["GeneGene_Hs"]["ir"].shape)
print("GeneGene jc: ", gp_data["GeneGene_Hs"]["jc"].shape)
print("Gene Features: ", gf_data["GeneFeatures"].shape) # 4536x12331
print("Clinical Features: ", cf_data["F"].shape) # 3215
print("GeneIds: ", gp_data["geneIds"].shape)
print("PheneIds: ", gp_data["pheneIds"].shape)
print(gp_data[gp_data["pheneIds"][0][0]][0]) # 1x3215
print("GenePhenes: ", gp_data["GenePhene"].shape)


gene_network_adj = sp.csc_matrix((np.array(gp_data['GeneGene_Hs']['data']),
    np.array(gp_data['GeneGene_Hs']['ir']), np.array(gp_data['GeneGene_Hs']['jc'])),
    shape=(12331,12331)).tocoo()

disease_network_adj = sp.csc_matrix((np.array(gp_data['PhenotypeSimilarities']['data']),
    np.array(gp_data['PhenotypeSimilarities']['ir']), np.array(gp_data['PhenotypeSimilarities']['jc'])),
    shape=(3215, 3215)).tocoo()

disease_offset = gene_network_adj.shape[0]+1


dg_ref = gp_data['GenePhene'][0][0]
gene_disease_adj = sp.csc_matrix((np.array(gp_data[dg_ref]['data']),
    np.array(gp_data[dg_ref]['ir']), np.array(gp_data[dg_ref]['jc'])),
    shape=(12331, 3215)).tocoo()

print(disease_offset)


print(gene_network_adj.shape)   # which genes are linked to each other
print(disease_network_adj.shape) # which diseases are linked to each other
print(gene_disease_adj.shape) # which genes are linked to which diseases

GeneGene data:  (733836,)
GeneGene ir:  (733836,)
GeneGene jc:  (12332,)
Gene Features:  (4536, 12331)
Clinical Features:  (16592, 3215)
GeneIds:  (1, 12331)
PheneIds:  (9, 1)
[2.00000e+00 5.00000e+00 1.00100e+05 ... 1.61550e+05 6.10805e+05
 6.14485e+05]
GenePhenes:  (9, 1)
12332
(12331, 12331)
(3215, 3215)
(12331, 3215)


In [29]:
# load up Gene Features into a tensor
gene_nodes = torch.tensor(gf_data["GeneFeatures"][:]).T
disease_nodes = torch.tensor(cf_data["F"][:]).T

gene_rows = gene_network_adj.row
gene_cols = gene_network_adj.col
gene_data = gene_network_adj.data

disease_rows = disease_network_adj.row
disease_cols = disease_network_adj.col
disease_data = disease_network_adj.data

gene_disease_rows = gene_disease_adj.row
gene_disease_cols =  gene_disease_adj.col
gene_disease_data = gene_disease_adj.data



gm_graph = dt.HeteroData()
gm_graph["gene"].x = gene_nodes

gm_graph["gene", "gene_gene", "gene"].edge_index = torch.tensor([gene_rows, gene_cols])
gm_graph["gene", "gene_gene", "gene"].edge_attr = torch.tensor(gene_data)

gm_graph["disease"].x = disease_nodes
gm_graph["disease", "dis_dis", "disease"].edge_index = torch.tensor([disease_rows, disease_cols])
gm_graph["disease", "dis_dis", "disease"].edge_attr = torch.tensor(disease_data)

gm_graph["gene", "gda", "disease"].edge_index = torch.tensor([gene_disease_rows, gene_disease_cols])
gm_graph["gene", "gda", "disease"].edge_attr = torch.tensor(gene_disease_data)



# gm_graph.add_edge_index(edge_index=[gene_rows, gene_cols], edge_attr=gene_data, source="gene", target="gene")
# gm_graph.add_edge_index(edge_index=[disease_rows, disease_cols], edge_attr=disease_data, source="disease", target="disease")
# gm_graph.add_edge_index(edge_index=[gene_disease_rows, gene_disease_cols], edge_attr=gene_disease_data, source="gene", target="disease")


gene_mutations = [gm_graph]


In [30]:
gm_graph.num_nodes
print(len(gm_graph.num_node_features))

2


In [31]:
from torch_geometric.data import InMemoryDataset

class GeneMutations(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, data_list=None):
        super(GeneMutations, self).__init__(root, transform, pre_transform)
        self.data, self.slices = self.collate(data_list)


# transform = T.Compose([
#     T.NormalizeFeatures(),
#     T.ToDevice(device),
#     T.RandomLinkSplit(num_val=0.05, 
#                       num_test=0.1, # uses 10% of data for test data set, rest is for training
#                       is_undirected=True,  
#                       add_negative_train_samples=False,
#                       edge_types=gm_graph.edge_types)
# ])

transform = T.Compose([
        T.ToUndirected(),
        T.RandomLinkSplit(
            num_val=0.05,
            num_test=0.1,
            is_undirected=True,
            neg_sampling_ratio=2.0,
            edge_types=gm_graph.edge_types,
            # rev_edge_types=("disease", "gda", "gene"),
            add_negative_train_samples=False
        )
    ]
)



gm = GeneMutations(".", transform=transform, data_list=gene_mutations)
train_data, val_data, test_data = gm[0]

print(train_data)
print(val_data)
print(test_data)

HeteroData(
  gene={ x=[12331, 4536] },
  disease={ x=[3215, 16592] },
  (gene, gene_gene, gene)={
    edge_index=[2, 623764],
    edge_attr=[623764],
    edge_label=[311882],
    edge_label_index=[2, 311882],
  },
  (disease, dis_dis, disease)={
    edge_index=[2, 2704872],
    edge_attr=[2704872],
    edge_label=[1352436],
    edge_label_index=[2, 1352436],
  },
  (gene, gda, disease)={
    edge_index=[2, 3362],
    edge_attr=[3362],
    edge_label=[3362],
    edge_label_index=[2, 3362],
  },
  (disease, rev_gda, gene)={
    edge_index=[2, 3954],
    edge_attr=[3954],
  }
)
HeteroData(
  gene={ x=[12331, 4536] },
  disease={ x=[3215, 16592] },
  (gene, gene_gene, gene)={
    edge_index=[2, 623764],
    edge_attr=[623764],
    edge_label=[55035],
    edge_label_index=[2, 55035],
  },
  (disease, dis_dis, disease)={
    edge_index=[2, 2704872],
    edge_attr=[2704872],
    edge_label=[238662],
    edge_label_index=[2, 238662],
  },
  (gene, gda, disease)={
    edge_index=[2, 3362],
   

In [31]:
# model = Net(graph.num_features, 128, 64)

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels=0):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), hidden_channels) # out_channels
    
    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x



class Classifier(torch.nn.Module):
    def decode(self, x_gene, x_disease, edge_label_index):
        edge_feat_gene = x_gene[edge_label_index[0]]
        edge_feat_disease = x_disease[edge_label_index[1]]
        return (edge_feat_gene * edge_feat_disease).sum(dim=-1)

    
class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin = torch.nn.Linear(20, hidden_channels)
        self.gene_emb = torch.nn.Embedding(gm_graph["gene"].num_nodes, hidden_channels)
        self.disease_emb = torch.nn.Embedding(gm_graph["disease"].num_nodes, hidden_channels)

        self.gnn = GNN(hidden_channels)
        self.gnn = to_hetero(self.gnn, metadata=gm_graph.metadata())

        self.classifier = Classifier()

    def decode_all(self, data):
        x_dict = {
            "gene": self.gene_emb(data["gene"].node_id),
            "disease": self.disease_emb(data["disease".node_id])
        }

        x_dict = self.gnn(x_dict, data.edge_index_dict)
        pred = self.classifier(
            x_dict["gene"],
            x_dict["disease"],
            data["gene", "gda", "disease"].edge_label_index
        )

        return pred


# model = GNN(hidden_channels=64, out_channels=len(gm_graph.num_node_features))
model = Model(hidden_channels=64)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception


In [388]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index,
        num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1),
        method='sparse'
    )

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1
    )

    edge_label = torch.cat(
        [train_data.edge_label, train_data.edge_label.new_zeros(neg_edge_index.size(1))],
        dim=0
    )

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss




@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [ ]:
best_val_auc = final_test_auc = 0
for epoch in range(1,101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, ' f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')
z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)